# The inventory

The inventory is arguably the most important piece of nornir. Everything revolves around it.

## Inventory Data

The inventory has three pieces of information:

* **hosts** - Hosts contain a mapping about each existing host, how to connect to them and may also contain user defined data. It also describes group membership and data may be inherited from parent groups or defaults.
* **groups** - Similar to hosts
* **defaults** - You can define here default values that will be used when a certain data point is not defined by a host or any of its parent groups.

## Understanding the Inventory by example

In [1]:
from nornir import InitNornir
nr = InitNornir(
    inventory={
        "options": {
            "host_file": "2_inventory/inventory/hosts.yaml",
            "group_file": "2_inventory/inventory/groups.yaml",
            "defaults_file": "2_inventory/inventory/defaults.yaml",
        }
    }
)

In [2]:
nr.inventory.hosts

{'spine00.bma': Host: spine00.bma,
 'spine01.bma': Host: spine01.bma,
 'leaf00.bma': Host: leaf00.bma,
 'leaf01.bma': Host: leaf01.bma,
 'spine00.cmh': Host: spine00.cmh,
 'spine01.cmh': Host: spine01.cmh,
 'leaf00.cmh': Host: leaf00.cmh,
 'leaf01.cmh': Host: leaf01.cmh}

In [3]:
nr.inventory.groups

{'bma': Group: bma,
 'cmh': Group: cmh,
 'eos': Group: eos,
 'junos': Group: junos}

In [4]:
nr.inventory.defaults

## Example: ``spine00.bma``

In [5]:
!sed '2,14!d' 2_inventory/inventory/hosts.yaml

spine00.bma:
    # well-known attributes; hostname, username, password, platform and port
    hostname: spine00.bma.acme.com
    platform: junos

    groups: # group membership
        - bma
        - junos

    data:  # user-defined data
        site: bma
        role: spine
        region: EU


In [6]:
!sed '2,4!d' 2_inventory/inventory/groups.yaml

bma:
    data:
        asn: 65100


In [7]:
!sed '14,16!d' 2_inventory/inventory/groups.yaml

junos:
    username: junos_auto
    port: 5022


In [8]:
!sed '2,3!d' 2_inventory/inventory/defaults.yaml

data:
    domain: acme.com


In [9]:
nr.inventory.hosts["spine00.bma"].hostname # comes from the device data

'spine00.bma.acme.com'

In [10]:
nr.inventory.hosts["spine00.bma"].username # comes from the group "junos"

'junos_auto'

In [11]:
nr.inventory.hosts["spine00.bma"]["asn"] # comes from the group "bma"
                                         # user-defined data is accessed in a dict-like fashion

65100

In [12]:
nr.inventory.hosts["spine00.bma"]["domain"] # comes from the defaults

'acme.com'

## Example: ``leaf01.bma``

In [13]:
!sed '39,49!d' 2_inventory/inventory/hosts.yaml

leaf01.bma:
    hostname: leaf01.bma.acme.com
    platform: eos
    groups:
        - bma
        - eos
    data:
        site: bma
        role: leaf
        region: EU
        asn: 65111


In [14]:
nr.inventory.hosts["leaf01.bma"].username # comes from the group "eos"

'eos_auto'

In [15]:
nr.inventory.hosts["leaf01.bma"]["asn"] # comes from the host itself this time

65111

Groups and defaults behave the same way:

In [16]:
nr.inventory.groups["junos"].username

'junos_auto'

In [17]:
nr.inventory.groups["bma"]["asn"]

65100

In [18]:
print(nr.inventory.defaults.password)

None


In [19]:
nr.inventory.defaults.data["domain"] # note that unlike hosts and groups
                                     # there is a data attribute here

'acme.com'

You can easily add/modify/delete data at runtime. Let's start by setting a default password:

In [20]:
print(nr.inventory.defaults.password)

None


In [21]:
print(nr.inventory.hosts["leaf01.bma"].password)

None


In [22]:
# To avoid storing the password in plain files or in the db we could
# store it in some vault, use ssh keys, env variables or we could ask the user
nr.inventory.defaults.password = input("Dear user, enter the password: ")

Dear user, enter the password: p4ssw0rd


In [23]:
print(nr.inventory.defaults.password)

p4ssw0rd


In [24]:
print(nr.inventory.hosts["leaf01.bma"].password)

p4ssw0rd


In [25]:
# to remove it
nr.inventory.defaults.password = None

Now, let's add user-defined data:

In [26]:
nr.inventory.hosts["leaf01.bma"]["custom_attr"] = "set at runtime"

In [27]:
nr.inventory.hosts["leaf01.bma"]["custom_attr"]

'set at runtime'

In [28]:
# to remove it
nr.inventory.hosts["leaf01.bma"]["custom_attr"] = None